<a href="https://colab.research.google.com/github/emerenan/analysing_police_activity/blob/master/police_activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [51]:
import pandas as pd

# Método para leitura dos dados
def readfile(path):
    """ Método para ler os dados do arquivo e retornar um dataframe.
    Args: \n
        path: localização do arquivo que será lido. \n

    Returns: \n
        Dataframe contendo os dados para analise.

    Raises: \n
        ParserError: retorna as linhas que não puderam ser lidas.
    """

    try:
        df = pd.read_csv(path, error_bad_lines=False, warn_bad_lines=True, parse_dates=True, infer_datetime_format=True)
        print(df.head())
        print('-----------------------------------------------------------------------')
        print(df.isnull().sum())
        print('-----------------------------------------------------------------------')
        print(df.info())
        print('-----------------------------------------------------------------------')
        
        return df
    except pd.io.common.ParserError:
        print("Dados não puderam ser lidos")

# lendo o arquivo e salvando na variavel data
data = readfile('/content/police.csv')

dict = {}
for label, entry in data.isnull().sum().items():
    # Salvando a forma do dataframe na variavel t
    t = data.shape
    # Salvando a qtd de linhas na variavel rows
    rows = t[0]

    if entry == rows or entry >= rows/2:
        data.drop(label, axis='columns', inplace=True)
        if label in dict.keys():
            dict[label] += "Deletada"
        else:
            dict[label] = "Deletada"

for key, value in dict.items():
    if value == 0:
        print("Não houve colunas deletadas!")
    else:
        print(key, ": ", value)

print(data.columns)

  state   stop_date stop_time  ...  stop_duration drugs_related_stop district
0    RI  2005-01-04     12:55  ...       0-15 Min              False  Zone X4
1    RI  2005-01-23     23:15  ...       0-15 Min              False  Zone K3
2    RI  2005-02-17     04:15  ...       0-15 Min              False  Zone X4
3    RI  2005-02-20     17:15  ...      16-30 Min              False  Zone X1
4    RI  2005-02-24     01:20  ...       0-15 Min              False  Zone X3

[5 rows x 15 columns]
-----------------------------------------------------------------------
state                     0
stop_date                 0
stop_time                 0
county_name           91741
driver_gender          5205
driver_race            5202
violation_raw          5202
violation              5202
search_conducted          0
search_type           88434
stop_outcome           5202
is_arrested            5202
stop_duration          5202
drugs_related_stop        0
district                  0
dtype: int64
----